In [1]:
# !pip3 install -q -U bitsandbytes==0.42.0  
# !pip3 install q U peft==0.8.2  
# !pip3 install -q U trl==0.7.10  
# !pip3 install -q-U accelerate==0.27.1  
# !pip3 install-q-U datasets==2.17.0  
# !pip3 install -q -U transformers==4.38.0  



!pip3 install bitsandbytes
!pip3 install peft
!pip3 install trl
!pip3 install accelerate
!pip3 install datasets
!pip3 install transformers
!pip install python-dotenv

In [2]:
import os
import transformers
import torch
# from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer
from dotenv import load_dotenv
import os
device = 'gpu' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

d:\Coursera Courses (NVME)\LLM freecodecamp\LLM-finetuning-LLM\fine-tuning\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: gpu


In [3]:
load_dotenv('token.env')  # path to your token.env file

# Step 3: Access the token
hf_token = os.getenv("HF_TOKEN")
print("Hugging Face Token:", hf_token[:10] + "..." if hf_token else "Token not found")


Hugging Face Token: hf_tLCDXEW...


In [4]:
!huggingface-cli login --token $hf_token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `LLM` has been saved to C:\Users\ZAIN\.cache\huggingface\stored_tokens
Your token has been saved to C:\Users\ZAIN\.cache\huggingface\token
Login successful.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


nf4(4-bit NormalFloat(NF4)): https://www.kaggle.com/code/lorentzyeung/what-s-4-bit-quantization-how-does-it-help-llama2


In [5]:
!huggingface-cli whoami

ghourimartin


In [6]:
model_id = "google/gemma-2b"  
bnb_config = BitsAndBytesConfig(load_in_4bit=True,  
                                bnb_4bit_quant_type="nf4",  
                                bnb_4bit_compute_dtype=torch.bfloat16  
)  

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token)  
model = AutoModelForCausalLM.from_pretrained(model_id,  
                                             quantization_config=bnb_config,  
                                             device_map={"":0},  
                                             token=hf_token)   

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.47s/it]


In [8]:
text = "Quote: Imagination is more,"
device = "cuda:0"
inputs = tokenizer (text, return_tensors="pt").to(device)
inputs   

{'input_ids': tensor([[     2,  14782, 235292, 122399,    603,    978, 235269]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [9]:
outputs = model.generate (**inputs, max_new_tokens=500)
#  

## **Data Import**

In [10]:
from datasets import load_dataset  
# Load the dataset
data = load_dataset("Abirate/english_quotes")  
data = data.map(lambda samples: tokenizer (samples ["quote"]), batched=True)  

In [11]:
print(tokenizer.decode (outputs [0], skip_special_tokens=True))

Quote: Imagination is more, than knowledge.

I am a self-taught artist, born in the Philippines. I am a mother of two beautiful children. I am a wife to a wonderful husband. I am a daughter to a loving father. I am a sister to a caring brother. I am a friend to many. I am a person who is always willing to help others. I am a person who is always willing to learn. I am a person who is always willing to grow. I am a person who is always willing to love. I am a person who is always willing to be kind. I am a person who is always willing to be a good example. I am a person who is always willing to be a good friend. I am a person who is always willing to be a good person. I am a person who is always willing to be a good person. I am a person who is always willing to be a good person. I am a person who is always willing to be a good person. I am a person who is always willing to be a good person. I am a person who is always willing to be a good person. I am a person who is always willing to 

In [12]:
print(f'data["train"].column_names)')
print(data['train'])
data['train']['quote'][0]

data["train"].column_names)
Dataset({
    features: ['quote', 'author', 'tags', 'input_ids', 'attention_mask'],
    num_rows: 2508
})


'“Be yourself; everyone else is already taken.”'

In [13]:
os.environ["WANDB_DISABLED"] = "false"

## **Training Parameters**

In [ ]:
def formatting_func(example):
    text = f"Quote: {example['quote'][0]}\nAuthor: {example['author'][0]}"
    return [text]

formatting_func(data)

In [15]:
lora_config = LoraConfig(r = 8,
                         target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                                           "gate_proj", "up_proj", "down_proj"],
                         task_type = "CAUSAL_LM",)

In [16]:
training_arguments = transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=50,
        max_steps=1000, #100,
        learning_rate=5e-5, # 2e-3,
        max_grad_norm=1.0,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="adamw_torch",
        lr_scheduler_type="cosine",
        weight_decay=0.01
    )

In [17]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data["train"],
    args=training_arguments,
    peft_config=lora_config,
    formatting_func=formatting_func,
)

d:\Coursera Courses (NVME)\LLM freecodecamp\LLM-finetuning-LLM\fine-tuning\Lib\site-packages\trl\trainer\sft_trainer.py:535: UserWarning: You passed a dataset that is already processed (contains an `input_ids` field) together with a formatting function. Therefore `formatting_func` will be ignored. Either remove the `formatting_func` or pass a dataset that is not already processed.
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [18]:
trainer.train()

Step,Training Loss
1,2.561000
2,1.626600
3,2.488800
4,2.767600
5,2.312800
6,2.499900
7,2.918900
8,2.309000
9,3.279500
10,2.335900


TrainOutput(global_step=1000, training_loss=1.9510655289888381, metrics={'train_runtime': 1263.3583, 'train_samples_per_second': 3.166, 'train_steps_per_second': 0.792, 'total_flos': 1967997092904960.0, 'train_loss': 1.9510655289888381})

## **Inference**

In [19]:
text = "I'm selfish, impatient"
device = "cuda:0"
inputs = tokenizer (text, return_tensors="pt").to(device)
outputs = model.generate (**inputs, max_new_tokens=120)

print(f"Input text: {inputs}\n")
print(f"Output text: {outputs}\n")
print("Generated text:\n")
print(tokenizer.decode(outputs [0], skip_special_tokens=True))


Input text: {'input_ids': tensor([[     2, 235285, 235303, 235262,  44166, 235269,  76694]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

Output text: tensor([[     2, 235285, 235303, 235262,  44166, 235269,  76694,    578,    476,
           2356,  93811, 235265,    590,   4564,    674,   1853,    576,    917,
            919,    476,   6239,   8135,    577,   3255,    573,   2134, 235269,
            578,    573,   2134,   4026,   1212,   1297,   1853,    576,    917,
            798,   2734, 235265,    109, 235285, 235303, 235262,    476,   2356,
          14565, 235265,    590, 235303, 235262,    476,   2356,   3298,    576,
            476,   1160, 235250,  10540, 235265,    590, 235303, 235262,    476,
           2356,   3298,    576,    476,  30945,    694, 235265,    590, 235303,
         235262,    476,   2356,   3298,    576,    476, 144932, 235265,    590,
         235303, 235262,    476,   2356,   3298,    576,    476,  23939